# Análisis de comportamiento de clientes (`customer_id`)

En este notebook exploramos los patrones de compra de los clientes. Aplicaremos un enfoque **Top-Down** y **Bottom-Up** para responder preguntas como:

- ¿Cuáles son los clientes más importantes por volumen?
- ¿Tienen patrones de compra estables o erráticos?
- ¿Hay clientes que solo compran ciertos meses o familias?

El objetivo es generar insights que luego puedan alimentar modelos o reglas de negocio.


## Carga de librerías

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact


## Top‑Down: Clientes con mayor volumen total

In [2]:

# Clientes con mayor volumen total
clientes_top = df_all.groupby('customer_id')['tn'].sum().sort_values(ascending=False).head(20)

clientes_top.plot(kind='bar', figsize=(10,4), color='teal')
plt.title('Top 20 clientes por toneladas compradas')
plt.ylabel('Toneladas')
plt.xlabel('ID Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


NameError: name 'df_all' is not defined

## Bottom‑Up: Evolución mensual de cada cliente (interactivo)

In [ ]:

# Tabla de toneladas por mes y cliente
pivot_clientes = df_all.pivot_table(index='periodo', columns='customer_id', values='tn', aggfunc='sum', fill_value=0)

@interact(cliente=sorted(pivot_clientes.columns))
def grafico_cliente(cliente):
    pivot_clientes[cliente].plot(figsize=(10,3))
    plt.title(f'Evolución mensual de compras - Cliente {cliente}')
    plt.ylabel('Toneladas')
    plt.xlabel('Periodo')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


## Estabilidad de compras: ¿Qué tan constantes son los clientes?

In [ ]:

# Coeficiente de variación por cliente
cv_clientes = pivot_clientes.std() / pivot_clientes.mean()
cv_clientes = cv_clientes.replace([np.inf, -np.inf], np.nan).dropna()

cv_clientes.hist(bins=30, figsize=(6,3))
plt.title('Distribución del Coef. de Variación entre clientes')
plt.xlabel('CV (Desvío / Media)')
plt.ylabel('Cantidad de clientes')
plt.grid(True)
plt.tight_layout()
plt.show()


## Preferencias por familia de producto (`cat1`)

In [ ]:

# Top clientes por familia
df_familia = df_all.groupby(['customer_id', 'cat1'])['tn'].sum().reset_index()
pivot_familia = df_familia.pivot(index='customer_id', columns='cat1', values='tn').fillna(0)

pivot_familia_norm = pivot_familia.div(pivot_familia.sum(axis=1), axis=0)  # normalizar

pivot_familia_norm.sample(20).plot(kind='bar', stacked=True, figsize=(12,5), colormap='tab20')
plt.title('Distribución de familias en 20 clientes (aleatorios)')
plt.ylabel('Proporción del total comprado')
plt.xlabel('Cliente')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
